# Hobbit Run
This is a game where a hobbit is chased around the screen by orcs. By clever dodging amongst the tree, the hobbit can get the orcs to kill themselves on the trees. At the higher levels, the hobbit gains the ability sometimes hide in the trees and attack orcs directly.

# Rules
The game will begin by placing trees and orcs on a square grid. The hobbit will be place in the middle of the grid, and any orcs in the immediate vicinity of the hobbit will be excluded. (Gotta give the hobbit a fair chance!) The hobbit can be moved around using the keyboard using any combination of these systems:

Numeric keypad:

    7 8 9

    4 5 6

    1 2 3
    
Arrows:

         UP

    LEFT    RIGHT

        DOWN
        
Alphabetic:

    y u i

    h j k

    n m ,
    
After the hobbit makes its move, the orcs will blindly move in the most direct way possible regardless of tree obstacles. When an orc moves into a tree square, it is eliminated. Often, more than one orc occupies a single square. This matters not a whit as far as affecting the hobbit's strategy as the orcs will all move as a group. 

Once all of the orcs are eliminated, the hobbit is promoted to gain greater chances of hiding in the trees or attacking (not avoid capture) orcs. Of course, as the hobbit get more capability the levels get harder. 

Once the hobit defeats all of the ocs on all levels, it becomes the king of the orcs. If the hobbit unsuccessfully hides in the trees, or or unsuccessfully attack an orc, or is caught by an orc, the hobbit starts over at the lowest level.

To quit the game, close the window.



# Program

This program is motivated by the ticking off an item on my bucket list, the rewriting of a favourite game. Almost 40 years ago, I bought a Commodore 64 (C64), but I had limited software development tools on it. I did have access to a cross assembler on a Digital DEC10, so I wrote a version of this game using the cross assembler for a 6502 CPU and downloaded it on a very slow modem (300 BAUD) to my C64. With the software tools in hand on my PC, I have now been able to write a much more concise version with more features than the original. In particular, the sprite collision code in [pygame](https://www.pygame.org/news) simplified the coding by quite a bit.

To run the code as is, all you need to do is to install Python, Jupyter, and pygame. You will also the graphics for the sprites: *tree.jpg, orc.jpg, hobbit.jpg*.

If there is sufficient demand, I might put the game up on [Binder](https://mybinder.org/).


## Imports

Firstly, we will import the built in modules random (for probabilistic decisions), collections (for nametupole to hold static level information), and math (for copysign function used in orc move calculations).

In [1]:
import random
from collections import namedtuple
from math import copysign
from time import sleep
# import os
# os.environ["SDL_VIDEODRIVER"] = "directfb"

We then import the third party module, pygame, because it has a number of nice collision detection routine using sprites. We also import from pygame definitions of the various keyboard key code we need when doing event processing.

There are three ways of navigating the hobbit around the board.

Numeric keypad:

    7 8 9

    4 5 6

    1 2 3
    
Arrows:

         UP

    LEFT    RIGHT

        DOWN
        
Alphabetic:

    y u i

    h j k

    n m ,

In [3]:
import pygame
from pygame.locals import (
    RLEACCEL,
    K_KP1,  # keypad 1, SW
    K_KP2,  # keypad 2, S
    K_KP3,  # keypad 3, SE
    K_KP4,  # keypad 4, W
    K_KP5,  # keypad 5, say put
    K_KP6,  # keypad 6, E
    K_KP7,  # keypad 7, NW
    K_KP8,  # keypad 8, N
    K_KP9,  # keypad 9, NE
    K_UP,  # up arrow, N
    K_DOWN,  # down arrow, S
    K_RIGHT,  # right arrow, E
    K_LEFT,  # left arrow, W
    K_COMMA, # comma, SE
    K_h,  # h, W
    K_i,  # i, NE
    K_j,  # j,stay put
    K_k,  # k, E
    K_m,  # m, S
    K_n,  # n, SW
    K_u,  # u, NW
    K_y,  # y, N
)
from pygame.locals import *

## Global Variables

Here we define a `namedtuple` to hold the parameters necessary to define the starting positions at each level.

- `title` - Rank of hobbit in campaign
- `tree_density` - probability of a cell being occupied by a tree
- `orc_density` - probability of a cell being occupied by an orc 
- `buff` - orc free distance between hobbit and orcs at start
- `hide_prob` - probability of successful hide in trees of hobbit
- `attack_prob` - probability of a successful attack of hobbit on an orc

These level parameters have not been been fine tuned! However, each level will be more difficult that the last.

In [4]:
Level = namedtuple("Level", "title tree_density orc_density buff hide_prob attack_prob")
levels = [
    Level("Novice", .05, .01, 4, 0, 0),
    Level("Woodsman", .05, .015, 4, .1, .1),
    Level("Tracker", .05, .02, 4, .2, .2),
    Level("Strider", .05, .02, 3, .3, .30),
    Level("Warrior", .05, .025, 3, .4, .4),
    Level("Elder", .05, .025, 2, .5, .5),
    Level("Chieftan", .05, .025, 2, .6, .6),
    Level("Hero", .05, .030, 2, .7, .7),
    Level("Legend", .05, .035, 2, .8, .8),
    Level("Wizard", .05, .035, 2, .9, .9),
]

Here we define a number of variables to describe our screen.

- `cells_per_edge` - number of cells per edge of our square playing area
- `cell_size` - size of cel in pixels (sprite gfx must conform!)
- `hob_pos` - initial position of hobbit in pixels (centre of screen)
- `edge` - size of playing edge in pixels
- `number_cells` - number of cells in playing area
- `background_colour` - background colour (kinda brown)

In [5]:
cells_per_edge = 19
cell_size = 36
hob_pos = int(cells_per_edge / 2) * cell_size
edge = cells_per_edge * cell_size
number_cells = cell_size * cell_size
background_colour = (165, 100, 100)

## Classes

Here we define a number of classes derived from pygame's sprite class. The imagery for all of these classes will all be of the same size with background to be set transparent (which my images are not yet doing that properly), and the rectangles of the sprite are defined as the same side as the image for purpose of calculating collisions defined as overlaps of the sprite image rectangles.

### Tree

This class defines a sprite repesenting a tree with imagery from *tree.jpg* with collision rectangle being cell sized. 

In [6]:
class Tree(pygame.sprite.Sprite):
    def __init__(self):
        super(Tree, self).__init__()
        self.surf = pygame.Surface((cell_size, cell_size))
        self.surf = pygame.image.load("tree.jpg")  # .convert()
        self.surf.set_colorkey((255, 255, 255), RLEACCEL)
        self.image = self.surf
        self.rect = self.surf.get_rect()

### Orc
This class defines a sprite repesenting an orc with imagery from *orc.jpg* with collision rectangle being cell sized. 

In [7]:
class Orc(pygame.sprite.Sprite):
    def __init__(self):
        super(Orc, self).__init__()
        self.surf = pygame.Surface((cell_size, cell_size))
        self.surf = pygame.image.load("orc.jpg")  # .convert()
        self.surf.set_colorkey((255, 255, 255), RLEACCEL)
        self.rect = self.surf.get_rect()
        self.image = self.surf

### Hobbit
This class defines a sprite repesenting the hobbit with imagery from *hobbit.jpg* with collision rectangle being cell sized. 

In [8]:
class Hobbit(pygame.sprite.Sprite):
    def __init__(self):
        super(Hobbit, self).__init__()
        self.surf = pygame.Surface((cell_size, cell_size))
        self.surf = pygame.image.load("hobbit.jpg")  # .convert()
        self.surf.set_colorkey((255, 255, 255), RLEACCEL)
        self.rect = self.surf.get_rect()
        self.image = self.surf

### CZ
This class defines a hidden sprite (it is never rendered upon the screen) which will be used at the start to clear orcs from the buffer zone surrounding the hobbit in the middle of the screen. The rectangle is defined in terms of screen coordinates rather than being read from a image file.

In [9]:
class CZ(pygame.sprite.Sprite):
    def __init__(self, buff):
        super(CZ, self).__init__()
        top = hob_pos - buff * cell_size
        left = hob_pos - buff * cell_size
        height = (2 * buff + 1) * cell_size
        right = (2 * buff + 1) * cell_size
        self.r = (top, left, height, right)
        self.surf = pygame.Surface((height, height))
        self.rect = self.surf.get_rect()
        self.rect.x = left
        self.rect.y = top

## Functions

### `msg()`
This function print the string, `current_msg` into the message aea below the playing area.

In [10]:
def msg():
    font = pygame.font.SysFont(None, cell_size - 4)
    img = font.render(current_msg, True, (255, 255, 0))
    screen.blit(img, (2, edge + 5))

### `refresh_screen()`
This function draw the screen and then displays it
- sets background colour
- draws the tree sprites
- draws the orc sprites
- draws the hobbit sprite
- optionally draws the boundary of teh buffer zone between the hobbit and orcs at the beginning of the scenario
- flips the diplay buffers

In [11]:
def refresh_screen(draw_buffer=True):
    screen.fill(background_colour)
    trees.draw(screen)
    orcs.draw(screen)
    hl.draw(screen)
    if draw_buffer:
        pygame.draw.rect(screen, (0, 0, 0), cz.r, 1)
    msg()
    pygame.display.flip()

## Game Initialization

- Initialize the `pygame` engine
- define the screen including a message display area below the playing area
- set the game player level, `level_index`, to 0

In [12]:
pygame.init()
screen = pygame.display.set_mode((edge, edge + cell_size))
level_index = 0

## Main Game Loop

This is the main logic to implement the hobbit's moves and the orcs' reactive moves.

In [13]:
running = True # flag that says continue the game
while running:
    while level_index < len(levels): 
        if not running:
            break
        # set up scenario for the level
        level = levels[level_index] # set the level of the game

        cz = CZ(level.buff)

        nt = int(level.tree_density * number_cells) # randomly scatter trees amongst the playing area
        trees = pygame.sprite.Group()
        for i in range(nt):
            j = random.randrange(0, cells_per_edge)
            k = random.randrange(0, cells_per_edge)
            tree = Tree()
            tree.rect.x = j * cell_size
            tree.rect.y = k * cell_size
            trees.add(tree)

        norcs = int(level.orc_density * number_cells) # randomly scatter orcs
        orcs = pygame.sprite.Group()
        for i in range(nt // 2):
            j = random.randrange(0, cells_per_edge)
            k = random.randrange(0, cells_per_edge)
            orc = Orc()
            orc.rect.x = j * cell_size
            orc.rect.y = k * cell_size
            orcs.add(orc)

        hl = pygame.sprite.Group() # put orc in centre of playing area
        hobbit = Hobbit()
        hobbit.rect.x = hob_pos
        hobbit.rect.y = hob_pos
        hl.add(hobbit)

        # kill all orcs in central zone around hobbit
        z = pygame.sprite.spritecollide(cz, orcs, dokill=True, collided=None)
        # kill tree if it is on hobbit's position
        z = pygame.sprite.spritecollide(hobbit, trees, dokill=True, collided=None)
        # kill orcs occupying same positions as trees
        z = pygame.sprite.groupcollide(orcs, trees, True, False)

        # define beginning scenario message
        current_msg = "%s :: %d Orcs" % (level.title,len(orcs))
        refresh_screen()
            
        # event loop for moves
        exec_level = True # level is still being executed 
        while exec_level:
            if not running:
                break
            for event in pygame.event.get():
                if event.type == pygame.QUIT: # test to see if user has closed pygame window
                    running=False
                elif event.type == pygame.KEYDOWN: # test for key strokes that would move hobbit
                    move_hobbit = False
                    ek = event.key
                    dx = dy = 0
                    if ek in [K_KP1,K_n]:
                        dx = -cell_size
                        dy = cell_size
                        move_hobbit = True
                    elif ek in [K_KP2, K_DOWN, K_m]:
                        dx = 0
                        dy = cell_size
                        move_hobbit = True
                    elif ek in [K_KP3, K_COMMA]:
                        dx = cell_size
                        dy = cell_size
                        move_hobbit = True
                    elif ek in [K_KP4, K_LEFT, K_h]:
                        dx = -cell_size
                        dy = 0
                        move_hobbit = True
                    elif ek in [K_KP5,K_j]:
                        dx = 0
                        dy = 0
                        move_hobbit = True
                    elif ek in [K_KP6, K_RIGHT, K_k]:
                        dx = cell_size
                        dy = 0
                        move_hobbit = True
                    elif ek in [K_KP7, K_y]:
                        dx = -cell_size
                        dy = -cell_size
                        move_hobbit = True
                    elif ek in [K_KP8, K_UP, K_u]:
                        dx = 0
                        dy = -cell_size
                        move_hobbit = True
                    elif ek in [K_KP9, K_i]:
                        dx = cell_size
                        dy = -cell_size
                        move_hobbit = True
                   
                    if move_hobbit: # hobbit is moving
                        hobbit.rect.x += dx
                        hobbit.rect.y += dy
                        x = hobbit.rect.x
                        y = hobbit.rect.y
                        # don't let hobbit out of playing area
                        if x < 0: 
                            hobbit.rect.x = 0
                        elif x >= edge:
                            hobbit.rect.x = edge - cell_size  
                        if y < 0:
                            hobbit.rect.y = 0
                        elif y >= edge:
                            hobbit.rect.y = edge - cell_size  
                        # see if hobbit moved into a tree and, if so, can it hide
                        if len(pygame.sprite.spritecollide(hobbit, trees, dokill=False)) > 0:
                            if random.random() >= level.hide_prob:
                                current_msg = "%s :: Your ran into a tree, restart in a few seconds" % (level.title,)
                                refresh_screen()
                                sleep(5)
                                level_index=0
                                exec_level = False
                            else:
                                current_msg = "%s :: Your successful hid in the trees, move on" % (level.title,)
                                refresh_screen()
                        # see if hobbit moved into an orc and, if so, can it kill it
                        elif len(pygame.sprite.spritecollide(hobbit, orcs, dokill=False, collided=None)) > 0:
                            if random.random() > level.attack_prob:
                                current_msg = "%s :: you ran into an orc, restart in a few seconds" % level.title
                                refresh_screen()
                                sleep(5)
                                level_index=0
                                exec_level = False  
                            else:
                                current_msg = "%s :: Your successful attacked orcs, move on" % (level.title,)
                                pygame.sprite.spritecollide(hobbit, orcs, dokill=True, collided=None)
                                refresh_screen()
                        else: # move orcs
                            for orc in orcs:
                                dx = x - orc.rect.x
                                dy = y - orc.rect.y
                                if abs(dx) > abs(dy):
                                    orc.rect.x += copysign(cell_size, dx)
                                elif abs(dy) > abs(dx):
                                    orc.rect.y += copysign(cell_size, dy)
                                else:
                                    orc.rect.x += copysign(cell_size, dx)
                                    orc.rect.y += copysign(cell_size, dy)
                            # kill orcs hat run into trees
                            pygame.sprite.groupcollide(orcs, trees, True, False) 
                            current_msg = "%s :: %d Orcs" % (level.title,len(orcs))
                            # see if orcs capture hobbit
                            if len(pygame.sprite.spritecollide(hobbit, orcs, dokill=False)) > 0:
                                current_msg = "%s :: The orcs caught you, restart in a few seconds" % (level.title,)
                                refresh_screen()
                                sleep(5)
                                level_index=0
                                exec_level = False
                            if len(orcs) == 0: # check to see if all orcs are dead
                                level_index += 1
                                if level_index < len(levels): # promote hobbit 
                                    level = levels[level_index]
                                    current_msg = ">>> You have been recognized as a %s" % (level.title,) 
                                    refresh_screen()
                                    sleep(5)
                                    exec_level = False
                                else: # conquered all levels
                                    current_msg = "Congratulations, you are crowned as the Hobbit King !!!"
                                    refresh_screen()
                                    sleep(5)
                                    level_index=0
                                    exec_level = False                      
                        refresh_screen() # display current state of play

# wrap up
pygame.display.quit()
pygame.quit()
exit()